In [1]:
%pip install scapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.3 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Created wheel for scapy: filename=scapy-2.5.0-py2.py3-none-any.whl size=1444328 sha256=2b57b156fb67a6710e196ba2bce81d894576ceaa06c9d9af31b2d3a87b108bca
  Stored in directory: /home/erlendsi/.cache/pip/wheels/11/ff/28/a1348d0c6d006242c053521732bc2951d0c9eca62a11430da1
Successfully built scapy
Note: you may need to restart the kernel to use updated packages.


In [13]:
from scapy.all import *
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)

In [24]:
def read_pcap(file_path):
    # Read the pcap file
    packets = rdpcap(file_path)
    count = 0

    # Process each packet
    for packet in packets:
        packet.show()
        count += 1
        if count > 10:
            break

if __name__ == "__main__":
    # Replace 'your_file.pcap' with the path to your .pcap file
    pcap_file_path = 'Modbus2023/benign/network-wide-normal-14.pcap'
    read_pcap(pcap_file_path)

###[ Ethernet ]### 
  dst       = 02:42:b9:af:00:03
  src       = 02:42:b9:af:00:04
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 60
     id        = 0
     flags     = DF
     frag      = 0
     ttl       = 64
     proto     = 6
     chksum    = 0xc756
     src       = 185.175.0.4
     dst       = 185.175.0.3
     \options   \
###[ TCP ]### 
        sport     = 502
        dport     = 37266
        seq       = 2717159107
        ack       = 3712470898
        dataofs   = 10
        reserved  = 0
        flags     = SA
        window    = 65160
        chksum    = 0x7394
        urgptr    = 0
        options   = [('MSS', 1460), ('SAckOK', b''), ('Timestamp', (1840849289, 49381357)), ('NOP', None), ('WScale', 7)]

###[ Ethernet ]### 
  dst       = 02:42:b9:af:00:04
  src       = 02:42:b9:af:00:03
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 52
     id        = 6

In [5]:
def pcap_to_dataframe(file_path):
    # Read the pcap file
    packets = rdpcap(file_path)

    # Extract relevant information from each TCP packet
    data = []
    for packet in packets:
        if IP in packet and TCP in packet:
            # Extract fields from IP and TCP layers
            packet_info = {
                'Source IP': packet[IP].src,
                'Destination IP': packet[IP].dst,
                'Protocol': 'TCP',
                'Source Port': packet[TCP].sport,
                'Destination Port': packet[TCP].dport,
                'Flags': packet.sprintf("%TCP.flags%"),
                'Summary': packet.summary(),
                # Add more fields as needed
            }
            data.append(packet_info)

    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(data)

    return df

if __name__ == "__main__":
    # Replace 'your_file.pcap' with the path to your .pcap file
    pcap_file_path = 'Modbus2023/benign/network-wide-normal-14.pcap'
    
    # Convert pcap data to a DataFrame
    dataframe = pcap_to_dataframe(pcap_file_path)

    # Display the DataFrame
    print(dataframe)

          Source IP Destination IP Protocol  Source Port  Destination Port  \
0       185.175.0.4    185.175.0.3      TCP          502             37266   
1       185.175.0.3    185.175.0.4      TCP        37266               502   
2       185.175.0.3    185.175.0.4      TCP        37266               502   
3       185.175.0.4    185.175.0.3      TCP          502             37266   
4       185.175.0.4    185.175.0.3      TCP          502             37266   
...             ...            ...      ...          ...               ...   
842822  185.175.0.3    185.175.0.5      TCP        60882               502   
842823  185.175.0.5    185.175.0.3      TCP          502             60882   
842824  185.175.0.5    185.175.0.3      TCP          502             60882   
842825  185.175.0.3    185.175.0.5      TCP        60882               502   
842826  185.175.0.3    185.175.0.5      TCP        60882               502   

       Flags                                            Summary

In [6]:
df_2 = pcap_to_dataframe('Modbus2023/attack/network-wide-normal-0.pcap')

In [17]:
with pd.option_context('display.max_colwidth', None):
    print(df_2["Summary"])

0                Ether / IP / TCP 185.175.0.3:58960 > 185.175.0.5:502 S
1               Ether / IP / TCP 185.175.0.5:502 > 185.175.0.3:58960 SA
2                Ether / IP / TCP 185.175.0.3:58960 > 185.175.0.5:502 A
3         Ether / IP / TCP 185.175.0.3:58960 > 185.175.0.5:502 PA / Raw
4                Ether / IP / TCP 185.175.0.5:502 > 185.175.0.3:58960 A
                                      ...                              
645528           Ether / IP / TCP 185.175.0.3:38482 > 185.175.0.8:502 R
645529          Ether / IP / TCP 185.175.0.8:502 > 185.175.0.3:38484 FA
645530           Ether / IP / TCP 185.175.0.3:38484 > 185.175.0.8:502 R
645531          Ether / IP / TCP 185.175.0.8:502 > 185.175.0.3:38486 FA
645532           Ether / IP / TCP 185.175.0.3:38486 > 185.175.0.8:502 R
Name: Summary, Length: 645533, dtype: object


In [18]:
df_2 = pcap_to_dataframe('Modbus2023/attack/substation-13.pcap')
with pd.option_context('display.max_colwidth', None):
    print(df_2["Summary"])

0                Ether / IP / TCP 185.175.0.3:36792 > 185.175.0.5:502 S
1               Ether / IP / TCP 185.175.0.5:502 > 185.175.0.3:36792 SA
2                Ether / IP / TCP 185.175.0.3:36792 > 185.175.0.5:502 A
3         Ether / IP / TCP 185.175.0.3:36792 > 185.175.0.5:502 PA / Raw
4                Ether / IP / TCP 185.175.0.5:502 > 185.175.0.3:36792 A
                                      ...                              
375104           Ether / IP / TCP 185.175.0.4:502 > 185.175.0.3:40930 A
375105           Ether / IP / TCP 185.175.0.3:40946 > 185.175.0.4:502 S
375106          Ether / IP / TCP 185.175.0.4:502 > 185.175.0.3:40946 SA
375107           Ether / IP / TCP 185.175.0.3:40946 > 185.175.0.4:502 A
375108    Ether / IP / TCP 185.175.0.3:40946 > 185.175.0.4:502 PA / Raw
Name: Summary, Length: 375109, dtype: object


In [ ]:
#read_pcap('Modbus2023/attack/veth291ab9a-13.pcap')
df_2 = pcap_to_dataframe('Modbus2023/attack/veth291ab9a-13.pcap')
with pd.option_context('display.max_colwidth', None):
    print(df_2["Summary"])

In [22]:
read_pcap('Modbus2023/attack/veth3efd353-8.pcap')
df_2 = pcap_to_dataframe('Modbus2023/attack/veth3efd353-8.pcap')
with pd.option_context('display.max_colwidth', None):
    print(df_2["Summary"])

Ether / IP / TCP 185.175.0.2:60686 > 185.175.0.5:502 A
Ether / IP / TCP 185.175.0.2:60686 > 185.175.0.5:502 FA
Ether / IP / TCP 185.175.0.5:502 > 185.175.0.2:60674 A
Ether / IP / TCP 185.175.0.2:60694 > 185.175.0.5:502 S
Ether / IP / TCP 185.175.0.5:502 > 185.175.0.2:60694 SA
Ether / IP / TCP 185.175.0.2:60694 > 185.175.0.5:502 A
Ether / IP / TCP 185.175.0.2:60694 > 185.175.0.5:502 PA / Raw
Ether / IP / TCP 185.175.0.5:502 > 185.175.0.2:60694 A
Ether / IP / TCP 185.175.0.5:502 > 185.175.0.2:60694 PA / Raw
Ether / IP / TCP 185.175.0.2:60694 > 185.175.0.5:502 A
Ether / IP / TCP 185.175.0.2:60694 > 185.175.0.5:502 FA
0                 Ether / IP / TCP 185.175.0.2:60686 > 185.175.0.5:502 A
1                Ether / IP / TCP 185.175.0.2:60686 > 185.175.0.5:502 FA
2                 Ether / IP / TCP 185.175.0.5:502 > 185.175.0.2:60674 A
3                 Ether / IP / TCP 185.175.0.2:60694 > 185.175.0.5:502 S
4                Ether / IP / TCP 185.175.0.5:502 > 185.175.0.2:60694 SA
             